In [1]:
import numpy as np
import matplotlib.pyplot as plt

From MML book

Definition 6.3 (Expected Value). The expected value of a function $g: \mathbb{R} \rightarrow$ $\mathbb{R}$ of a univariate continuous random variable $X \sim p(x)$ is given by
$$
\mathbb{E}_{X}[g(x)]=\int_{\mathcal{X}} g(x) p(x) \mathrm{d} x
$$
Correspondingly, the expected value of a function $g$ of a discrete random variable $X \sim p(x)$ is given by
$$
\mathbb{E}_{X}[g(x)]=\sum_{x \in \mathcal{X}} g(x) p(x)
$$

From Nando De Freitas https://www.youtube.com/watch?v=mz3j59aJBZQ&list=PLE6Wd9FR--EdyJ5lbFl8UuGjecvVw66F6&index=16&t=1957s
    
starting 50 mins

$I = \int_{\mathcal{X}} g(x)p(x)dx = \dfrac{\sum_{i=1}^N g(x_i)}{N}$ where $x_i \sim p(x)$ is a sample from p(x)

Example

$X \sim \mathcal{N}(0, 1)$

$g(x) = x$

$\mathbb{E}_{X}[g(x)] = \mu = 0$

Case II

$g(x) = x^2$


Now, we know that

$\mathbb{V}_{X}[x]=\mathbb{E}_{X}\left[x^{2}\right]-\left(\mathbb{E}_{X}[x]\right)^{2}$

In [13]:
N = 1000000
np.random.seed(0)
samples = np.random.normal(loc=0, scale=1, size=N)

mu_hat = np.mean(samples)
print(mu_hat)


exp_x2 = np.mean(np.square(samples))
print(exp_x2)

var = np.var(samples)
print(var)

0.0015121465155362318
0.9998449448099948
0.9998426582229105


We can similarly approximate the marginal likelihood as follows:

Marginal likelihood = $\int_{\mathcal{\theta}} P(D|\theta) P(\theta)d\theta = I = \dfrac{\sum_{i=1}^N P(D|\theta_i)}{N}$ where $\theta_i$ is drawn from $p(\theta)$

To do:
    
Linear regression in say two variables. Prior is $p(\theta)\sim \mathcal{N}([0, 0]^T, I)$. We can easily draw samples from this prior then the obtained sample can be used to calculate the likelihood. The marginal likelihood is the empirical mean of likelihoods derived in this way.